In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from collections import namedtuple
from statistics import mean
import itertools

In [2]:
df_pie = pd.read_csv("/gpfsdswork/projects/idris/sos/ssos039/bench_pie/bench_pie.csv")
print(len(df_pie))
df_pie.tail()

44


,typeGPU,nbrGPU,model_name,training_dist,stage,batch_size,valid_batch_size,seq_length,nb_layer_freezed,epoch_duration,train_tokens_s,val_tokens_s,maxMemAlloc,maxMemRes,mean_loss,perplexity
39,NVIDIA A100-SXM4-80GB,16,meta-llama/Llama-2-13b-hf,deepspeed,2,1,4,1024,0,601.811961,2125.679688,40907.515625,54.80,78.47,0.649376,1.677641
40,NVIDIA A100-SXM4-80GB,16,meta-llama/Llama-2-13b-hf,fsdp,1,2,4,1024,0,1040.999917,4665.710449,3008.703369,49.15,72.31,0.741164,1.757858
41,NVIDIA A100-SXM4-80GB,16,meta-llama/Llama-2-13b-hf,fsdp,1,3,4,1024,0,1204.425241,4493.056641,5911.994141,70.62,78.52,0.730055,1.743416
42,NVIDIA A100-SXM4-80GB,16,meta-llama/Llama-2-13b-hf,deepspeed,3,1,4,1024,0,861.615060,2533.576172,1305.801270,36.39,56.93,0.655871,1.682142
43,NVIDIA A100-SXM4-80GB,16,meta-llama/Llama-2-13b-hf,deepspeed,3,2,4,1024,0,1061.325204,3918.656006,2789.394043,58.93,78.48,0.634994,1.641071


In [3]:
def table_all_best(stats = "train_tokens_s"):
    
    df_pie = pd.read_csv("/gpfsdswork/projects/idris/sos/ssos039/bench_pie/bench_pie.csv")
    
    list_columns = ["model_name", "nbrGPU", "training_dist", "stage", "seq_length", "bs_1", "bs_2", "bs_3", "bs_4", "mem_1", "mem_2", "mem_3", "mem_4"]
    
    list_model_name = ["meta-llama/Llama-2-7b-hf", "meta-llama/Llama-2-13b-hf"]
    list_gpu = [4, 8, 16]
    list_training_dist = ["deepspeed","fsdp"]
    list_stage = [1, 2, 3]
    list_maxLength = [1024]
    list_bs = [1, 2, 3, 4]
    

    # Fill the data as wanted
    data = []
    
    for elem_model_name, elem_gpu, elem_training_dist, elem_stage, elem_maxLength in itertools.product(list_model_name, list_gpu, list_training_dist, list_stage, list_maxLength):
        
        list_temp = [elem_model_name, str(elem_gpu), elem_training_dist, str(elem_stage), str(elem_maxLength)]
        list_memRes = []
        check = False
        
        for elem_bs in list_bs:
            
            df_temp = df_pie[(df_pie.model_name == elem_model_name)&
                         (df_pie.nbrGPU == elem_gpu)&
                         (df_pie.training_dist == elem_training_dist)&
                         (df_pie.stage == elem_stage)&
                         (df_pie.seq_length == elem_maxLength)&
                         (df_pie.batch_size == elem_bs)].sort_values(by=[stats], ascending=[False])
            
            df_temp = df_temp[["maxMemRes", stats]]
            
            if len(df_temp) != 0:
                df_row = df_temp.iloc[0]
                value = int(df_row[stats].item())
                memRes = str(df_row['maxMemRes'].item())
                check = True
            else:
                value = int(0)
                memRes = str(0)
                
            list_memRes.append(memRes)
            list_temp.append(value)
        
        if check:
            list_temp.extend(list_memRes)
            data.append(list_temp)
    
    # Create the datarfame
    df_final = pd.DataFrame(data, columns = list_columns)
    df_final = df_final.sort_values(by=["model_name", "nbrGPU", "training_dist", "stage"])
    df_final = df_final.style.background_gradient(cmap="RdYlGn", axis = 'columns')     
    return df_final

In [4]:
table_all_best(stats = "train_tokens_s")

,model_name,nbrGPU,training_dist,stage,seq_length,bs_1,bs_2,bs_3,bs_4,mem_1,mem_2,mem_3,mem_4
12,meta-llama/Llama-2-13b-hf,16,deepspeed,2,1024,2125,0,0,0,78.47,0,0,0
13,meta-llama/Llama-2-13b-hf,16,deepspeed,3,1024,2533,3918,0,0,56.93,78.48,0,0
14,meta-llama/Llama-2-13b-hf,16,fsdp,1,1024,0,4665,4493,0,0,72.31,78.52,0
9,meta-llama/Llama-2-13b-hf,4,fsdp,1,1024,6256,2486,0,0,72.45,78.08,0,0
10,meta-llama/Llama-2-13b-hf,8,deepspeed,3,1024,5840,0,0,0,77.58,0,0,0
11,meta-llama/Llama-2-13b-hf,8,fsdp,1,1024,12274,15206,0,0,57.76,76.79,0,0
6,meta-llama/Llama-2-7b-hf,16,deepspeed,2,1024,0,9039,12595,12216,0,62.09,76.29,78.47
7,meta-llama/Llama-2-7b-hf,16,deepspeed,3,1024,0,9707,8075,17039,0,53.12,67.58,78.16
8,meta-llama/Llama-2-7b-hf,16,fsdp,1,1024,0,11092,16401,16915,0,45.19,62.12,75.75
0,meta-llama/Llama-2-7b-hf,4,deepspeed,2,1024,5832,0,0,0,69.25,0,0,0


In [5]:
table_all_best(stats = "val_tokens_s")

,model_name,nbrGPU,training_dist,stage,seq_length,bs_1,bs_2,bs_3,bs_4,mem_1,mem_2,mem_3,mem_4
12,meta-llama/Llama-2-13b-hf,16,deepspeed,2,1024,40907,0,0,0,78.47,0,0,0
13,meta-llama/Llama-2-13b-hf,16,deepspeed,3,1024,1305,2789,0,0,56.93,78.48,0,0
14,meta-llama/Llama-2-13b-hf,16,fsdp,1,1024,0,3008,5911,0,0,72.31,78.52,0
9,meta-llama/Llama-2-13b-hf,4,fsdp,1,1024,5641,11469,0,0,72.45,78.08,0,0
10,meta-llama/Llama-2-13b-hf,8,deepspeed,3,1024,6339,0,0,0,77.58,0,0,0
11,meta-llama/Llama-2-13b-hf,8,fsdp,1,1024,10145,19155,0,0,57.76,76.79,0,0
6,meta-llama/Llama-2-7b-hf,16,deepspeed,2,1024,0,87561,91519,96386,0,62.09,76.29,78.47
7,meta-llama/Llama-2-7b-hf,16,deepspeed,3,1024,0,6462,5664,15799,0,53.12,67.58,78.16
8,meta-llama/Llama-2-7b-hf,16,fsdp,1,1024,0,6759,11278,16441,0,45.19,62.12,75.75
0,meta-llama/Llama-2-7b-hf,4,deepspeed,2,1024,20047,0,0,0,69.25,0,0,0
